<a href="https://colab.research.google.com/github/Jumponpatha/portfolio/blob/main/Food_retail_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food Retail Analysis with Market Basket Anlaysis**

**Problem Statement** :
- What can be done to improve the purchased of products from the food retail?
- Which products can be the target for cross-selling?


## **Data Collection**

**Import Libraries**

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import mlxtend
from mlxtend.frequent_patterns import association_rules, apriori

**Load Dataset**

* *Transactions Dataset*

In [ ]:
transactions = pd.read_csv('dh_transactions.csv')
transactions

* *Product Detail Dataset*

In [ ]:
product = pd.read_csv('dh_product_lookup.csv')
product

* *Store Dataset*

In [ ]:
store = pd.read_csv('dh_store_lookup.csv')
store

* *Causal Dataset*

In [ ]:
causal = pd.read_csv('dh_causal_lookup.csv')
causal

## **Data Description**

In [ ]:
# Transactions dataset
transactions.info()

In [ ]:
# Product dataset
product.info()

In [ ]:
# Store dataset
store.info()

In [ ]:
# Causal dataset
causal.info()

## **Data Preparation**

* **Data Cleansing in Transactions Dataset**

1. Finding and Removing Null/NAN/missing value

In [ ]:
transactions.isnull().sum()



2. Find and removing Duplicated Columns

In [ ]:
transactions.duplicated().sum()

3. Finding and Removing with negative value

In [ ]:
# Show the non-positive 'dollar_sales'
transactions[transactions['dollar_sales'] < 0]

In [ ]:
# Selecting only positive 'dollar sales' ( More than 0)
transactions = transactions[transactions['dollar_sales'] > 0]
transactions

* **Converting a 'time of transaction' to Timestamp index**

In [ ]:
# Show 'time_of_transaction' column
pd.DataFrame(transactions['time_of_transaction'])

In [ ]:
# Transform 'time_of_transaction' column to Time index
transactions['time_of_transaction'] = transactions['time_of_transaction'].astype(str).str.zfill(4)
transactions['time_of_transaction'] = pd.to_datetime(transactions['time_of_transaction'], format='%H%M').dt.strftime('%H:%M')
transactions

In [ ]:
# Create columns and focus on 'hour'
transactions[['hour', 'minute']] = transactions['time_of_transaction'].str.split(':', expand=True)
transactions_new = transactions.drop(columns=['minute'])
transactions_new

* **Selecting relevant geographical areas for analysis by checking how all the
transactions are distributed**

In [ ]:
# Show frequency each geography area
pd.DataFrame(transactions.geography.value_counts(normalize=True))

In [ ]:
# Regarding geography area 1
geo_transaction = transactions[transactions['geography'] == 1]
geo_transaction

* **Selecting 'store' for analysis by checking how all the
transactions are distributed by 'store'**

In [ ]:
# List the most percentage frequency store
pd.DataFrame(geo_transaction.store.value_counts(normalize=True)[:10])

In [ ]:
# Select only regarding store
store_transactions = geo_transaction[geo_transaction['store'] == 71]
store_transactions

* **Remove all 'basket' that have only one item since we are analyzing items
bought together**

In [ ]:
# Define basket size
basket = store_transactions.groupby('basket').size()

# Select only multiple 'basket'
condition = basket > 1

# Get transactions data to new
new_transactions = transactions[transactions['basket'].isin(basket[condition].index)]
new_transactions

**Explanation:** The conclusion of data preparation, I had checking that no have Null/NaN/missing value and duplicated colums, selecting only non-negative value of 'dollars_sale', convert 'time_of_transaction' from integer to be time index and create 'hour' columns, regarding selected geography and store, and remove single 'basket' transactions. A total of 19,538 rows (roughly 1.035% of the initial count) were discarded from 5,197,681 rows.

## **Exploratory Data Analysis ( EDA )**

* **What items do customers buy more often?**

In [ ]:
# Ranking Percentage most popular products
sum_basket = transactions['basket'].count()
pd.DataFrame((transactions.groupby('upc')['basket'].count() / sum_basket).sort_values(ascending=False)[:10])

In [ ]:
# Ranking most popular products
pd.DataFrame((transactions.groupby('upc')['basket'].count()).sort_values(ascending=False)[:10])

In [ ]:
# Plot a bar chart
((transactions.groupby('upc')['basket'].count()).sort_values(ascending=False)[:10]).plot(kind='bar')

plt.xlabel('Record of Month')
plt.ylabel('Count of basket')
plt.title('The amount of product which customer buy per trip')
plt.grid(True)
plt.show()

**Result:** Top 10 most sold products represent around 13% of total items sold by the company.

* **What is the most popular store?**

1. Ranking percentage frequency of customer each store

In [ ]:
pd.DataFrame(transactions.store.value_counts(normalize=True).head(10))

2. Ranking frequency of customer each store

In [ ]:
(transactions.groupby('store')['basket'].nunique().sort_values(ascending=False).head(10)).plot(kind='bar')

plt.xlabel('Store')
plt.ylabel('Count of transaction')
plt.title('The frequency distribution of transaction by store')
plt.grid(True)
plt.show()

**Result:**
Top 10 most popular store which customer often to go.

* **What time of day would customers buy more often?**

In [ ]:
transactions_new.groupby('hour')['basket'].nunique().plot()
plt.xlabel('Hour')
plt.ylabel('Number of unique transactions')
plt.title('The frequency of Unique Baskets per Hour')
plt.grid(True)
plt.show()

**Result:** Majority of the transactions of occure between 10am. to 8pm. with the peak at a 5pm.

* **How much the total revenue by week?**

1. Rank the most income revenue each week record

In [ ]:
pd.DataFrame(transactions.groupby('week')['dollar_sales'].sum().sort_values(ascending=False)[:10])

2. Rank the least income revenue each week record

In [ ]:
pd.DataFrame(transactions.groupby('week')['dollar_sales'].sum().sort_values(ascending=True)[:10])

3. Record of income revenue each week

In [ ]:
transactions.groupby('week')['dollar_sales'].sum().plot(legend=True)
plt.grid(True)

**Result:** Sales by week is highly fructuated.

* **How much the sales product by every month record?**

In [ ]:
transactions['Month'] = (transactions['day'] // 30 ) + 1
transactions.groupby('Month')['dollar_sales'].sum().plot(legend=True)
plt.xlabel('Record of Month')
plt.ylabel('Count of basket')
plt.title('The amount of product which customer buy per trip')
plt.grid(True)
plt.show()

**Result:** Sales by month is however relatively steady between 350,000 to 450,000 dollars.

* **What is the ratio geography by transaction?**

The diagram represent ratio between geography 1 and 2.

In [ ]:
# Represent a ratio between geography 1 and 2 by trip
pd.DataFrame(transactions.geography.value_counts(normalize=True))

In [ ]:
# The chart shows percentage between geography 1 and 2 by trip
(transactions.geography.value_counts(normalize=True)).plot(kind='pie', x='geography', autopct='%0.00f%%', startangle=60)
plt.title('The ratio of area')
plt.show()

**Result:** Around 54% of total transactions is from area 1.

**How many product does each customer buy?**

1. The frequency the quantity of items

In [ ]:
select_col = transactions[['basket','upc']]
select_trans = select_col.pivot_table(index='basket', values='upc', aggfunc='count')

select_trans['upc'].plot(kind='hist', bins=30, title='upc')
plt.gca().spines[['top', 'right',]].set_visible(False)

plt.xlabel('amount of purchased')
plt.ylabel('Count of basket')
plt.title('The frequency of items per purchased.')
plt.grid(True)
plt.show()

**Note:** The 1e6 is represent a 1,000,000

2. The frequency percentage the quantity of items

In [ ]:
counts = select_trans['upc'].value_counts()
percentages = counts / counts.sum() * 100
result_df = pd.DataFrame({'Count': counts, 'Percentage': percentages})
result_df

**Result:** The large majority of customers(around 96%) typically purchased between 1 to 4 items, with a peak at 1 items(around 61%). There is a huge opportunity to identify items that can be potentially purchased with.

## **Apriori algorithm**

In [ ]:
# Create new dataframe from selected columns
df = new_transactions[['basket', 'upc', 'units']]
df

In [ ]:
# Group value using pivot table
transactions_pivot = df.pivot_table(index='basket', columns='upc', values='units', aggfunc='sum').fillna(0).astype(int)
transactions_pivot

**Note:** Each row corresponds to a unique 'basket,' each column corresponds to a unique 'upc', and the values represent the total count of 'units' purchased for each combination of basket and product.

In [ ]:
# Create a function for encode
def encode(item_freq):
    res = 0
    if item_freq > 0:
        res = 1
    return res

In [ ]:
# Get transactions_pivot to apply encode (0,1)
transactions_encoding = transactions_pivot.applymap(encode)
transactions_encoding

In [ ]:
# Apriori Rule
frequent_product = apriori(transactions_encoding, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_product, metric = "lift")
rules

**Note:** A frequent itemset is a collection of items that have a higher frequency of occurrence compared to the threshold value specified in the transaction. This value is also known as the minimum support.

**Note:**
Due to my output is empty, it may indicate that there are no frequent itemsets that meet the specified minimum support threshold of 0.01 in dataset. It's possible that the transactions in the dataset do not exhibit frequent patterns based on the chosen minimum support.

## **Apriori Agorithm by Association Rule**

Generate association rules from frequent itemsets with their
corresponding Support, Confidence, and Lift. From this we can then extract useful information about which items are more likely to be purchased together.


**Note:** To generate the frequent pattern with minimum support is 0.001

In [ ]:
# Set frequent pattern with minimum support is 0.001
frequent_product = apriori(transactions_encoding, min_support=0.001, use_colnames=True)
# Summary Support, Confidence and Lift with sort value by Lift
metric = 'lift'
min_threshold = 1

rules = association_rules(frequent_product, metric=metric, min_threshold=min_threshold)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules = rules.sort_values('lift', ascending=False).reset_index(drop=True)
rules.head(10)

## **Interpretation**

We can inteprete data from the table as followings,
* There is a strong support of 0.1164% transactions for upc: 7130030013 and  upc: 7130030011 in the same basket. We have 34.6154% confidence that upc: 7130030011 will be bought whenever upc: 7130030013 is bought. The purchased of product upc: 7130030011 is 70.45 times more whenever upc: 7130030013 is purchased than when upc: 7130030011 is purchased alone.

* There is a strong support of 0.1164% transactions for upc: 7130030011 and  upc: 7130030013 in the same basket. We have 23.6842% confidence that upc: 7130030011 will be bought whenever upc: 7130030011 is bought. The purchased of product upc: 7130030011 is 70.45 times more whenever upc: 7130030011 is purchased than when upc: 7130030013 is purchased alone.

* There is a strong support of 0.1552% transactions for upc: 4112900005, and  upc: 4112900006 in the same basket. We have 23.0769% confidence that upc: 4112900006 will be bought whenever upc: 4112900005 is bought. The purchased of product upc: 4112900006 is 55.77 times more whenever upc: 4112900005 is purchased than when upc: 4112900006 is purchased alone.

* There is a strong support of 0.1552% transactions for upc: 4112900006, and  upc: 4112900005 in the same basket. We have 37.5000% confidence that upc: 4112900006 will be bought whenever upc: 4112900005 is bought. The purchased of product upc: 4112900006 is 55.77 times more whenever upc: 4112900005 is purchased than when upc: 4112900006 is purchased alone.

* There is a strong support of 0.1164% transactions for upc: 7680850295, upc: 7151800016, and upc: 601011296 in the same basket. We have 6.97% confidence that upc: 7680850295 will be bought whenever upc: 7151800016,  and upc: 601011296 is bought. The purchased of product upc: 7130030016 is 49.05 times more whenever upc: 7680850295 is purchased than when upc: 7151800016,  and 601011296 is purchased alone.

* There is a strong support of 0.1164% transactions for upc: 7680850295, upc: 7151800016, and upc: 601011296 in the same basket. We have 81.8182% confidence that upc: 7151800016, and upc: 601011296 will be bought whenever upc: 7680850295 is bought. The purchased of product upc: 7130030011 is 49.05 times more whenever upc: 7680850295 is purchased than when upc: 7151800016,  and 601011296 is purchased alone.

* There is a strong support of 0.1939% transactions for upc: 601011296, upc: 7680850295 and  upc: 1800000956 in the same basket. We have 30.61% confidence that upc: 1800000956 will be bought whenever upc: 601011296 and upc: 7680850295 is bought. The purchased of product upc: 1800000956 is 45.52 times more whenever upc: 601011296 and upc: 7680850295 is purchased than when upc: 1800000956 is purchased alone.

* There is a strong support of 0.1939% transactions for upc: 601011296, upc: 7680850295 and  upc: 1800000956 in the same basket. We have 28.84% confidence that upc: 601011296 and upc: 7680850295 will be bought whenever upc: 1800000956 is bought. The purchased of product upc: 1800000956 is 45.52 times more whenever upc: 1800000956 is purchased than when upc: 601011296 and upc: 7680850295 is purchased alone.

* There is a strong support of 0.1164% transactions for upc: 7151800016, upc: 7680850295 and upc: 601011296 in the same basket. We have 4.76% confidence that  upc: 7151800016 and upc: 7680850295 will be bought whenever upc: 601011296 is bought. The purchased of product upc: 601011296 is 36.82 times more whenever upc: 7151800016 and upc: 7680850295 is purchased than when upc: 7151800016 and upc: 7680850295 is purchased alone.

* There is a strong support of 0.1164% transactions for upc: 7151800016, upc: 7680850295 and upc: 601011296 in the same basket. We have 90% confidence that upc: 601011296 will be bought whenever upc: 7151800016 and upc: 7680850295 is bought. The purchased of product upc: 601011296 is 36.82 times more whenever upc: 7151800016 and upc: 7680850295 is purchased than when upc: 601011296 is purchased alone.

As we set the cofindence threshold to at least 25%, so the association rules below 25% will be excluded. As a result, there are only 6 itemsets suitable for cross-selling.

* (7130030013) will be bought whenever (7130030011) is bought, (34.61%)

* (4112900006) will be bought whenever (4112900005) is bought, (37.50%)

* (7151800016, 601011296) will be bought whenever (7680850295) is bought, (81.81%)

* (601011296, 7680850295) will be bought whenever (1800000956) is bought, (30.61%)

* (1800000956) will be bought whenever(601011296, 7680850295) is bought, (28.84%)

* (7151800016, 7680850295) will be bought whenever (601011296) is bought, (90%)


## **Implication and Recommendation**



Due to time constrainted, I can only run the association rule analysis to identify the items that are frequently purchased together which is potential for cross-selling. In order to improve the purchase of product from the food retail, I recommend the store manager to rearrange the placement of each pair of product items mentioned above next to each other. I would recommend the further anlaysis to improve promotion strategies based on the dataset in the causal file.